In [6]:
import json
import textwrap
import pytrends
from pytrends.request import TrendReq
import nltk
from nltk.corpus import wordnet
import random
from tqdm import tqdm
from nltk.corpus import wordnet
from nltk.metrics.distance import edit_distance
from nltk.corpus import stopwords
# PlaintextParser
from sumy.parsers.plaintext import PlaintextParser
# LexRankSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
# LsaSummarizer
from sumy.summarizers.lsa import LsaSummarizer
# define Tokenizer
from sumy.nlp.tokenizers import Tokenizer
# define Stemmer
from sumy.nlp.stemmers import Stemmer
# define language
pytrend = TrendReq(hl='en-US', tz=360)
stopwords = nltk.corpus.stopwords.words('english')


import os
from PIL import Image, ImageDraw, ImageFont
# with open("ppn_deck_cleaned.json", "w") as write_file:
#     json.dump(card_deck, write_file, indent=4)

# read card_deck from ppn_deck.json file
# with open("ppn_deck.json", "r") as read_file:
    # card_deck = json.load(read_file)
with open('ppn_deck.json', 'r') as read_file:
    card_deck = json.load(read_file)

# clear the card_images folder
print("Clearing card_images folder...")
for filename in os.listdir("card_box"):
    os.remove(os.path.join("card_box", filename))



def summarize_text(text, num_sentences):
    """
    Summarize the given text using the LSA or LexRank summarization algorithms and return the summary as a string
    """
    # create a PlaintextParser object to parse the text
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    # choose a summarization algorithm
    # algorithm = LsaSummarizer()
    algorithm = LexRankSummarizer()

    # summarize the text and return the summary as a string
    summary = algorithm(parser.document, num_sentences)
    summary_text = "\n".join([str(sentence) for sentence in summary])

    return summary_text

def generate_card(title, definition, points, name=None):
    # create a blank image
    image = Image.new('RGB', (550, 850), (255, 255, 255))
    draw = ImageDraw.Draw(image)
    
    # set font sizes and create font objects
    title_font_size = 20
    description_font_size = 20
    points_font_size = 18
    title_font = ImageFont.truetype('./fonts/SFNSMono.ttf', title_font_size)
    description_font = ImageFont.truetype('./fonts/SFNSMono.ttf', description_font_size)
    points_font = ImageFont.truetype('./fonts/SFNSMono.ttf', points_font_size)
    
    # wrap the title and definition to 40 characters
    title_wrapped = textwrap.wrap(title, width=40)
    definition_wrapped = textwrap.wrap(definition, width=40)
    
    # calculate the heights of the title and definition text areas
    title_height = len(title_wrapped) * title_font_size
    description_height = len(definition_wrapped) * description_font_size
    
    # draw the title and definition text areas
    draw.rectangle([(10, 10), (540, 10 + title_height)], fill='lightblue')
    draw.rectangle([(10, 30 + title_height), (540, 30 + title_height + description_height)], fill='white')
    
    # write the title and definition text
    y_text = 20
    for line in title_wrapped:
        draw.text((270, y_text), line, fill=(0, 0, 0), font=title_font, anchor='mm')
        y_text += title_font_size
    y_text = 30 + title_height
    for line in definition_wrapped:
        draw.text((10, y_text), line, fill=(0, 0, 0), font=description_font, anchor='lm')
        y_text += 20
    
    # draw the points text area at the bottom left corner of the card
    draw.rectangle([(10, 830), (100, 850)], fill='lightgreen')

    # write the points text
    draw.text((55, 840), str(points), fill=(0, 0, 0), font=points_font, anchor='mm')


    
    # if a name is provided, write it in the bottom right corner
    if name:
        draw.text((540, 830), name, fill=(0, 0, 0), font=points_font, anchor='mm')
    
    # save the image
    title = title.replace(" ", "_").lower()
    # remove any non-alphanumeric characters
    title = "".join([char for char in title if char.isalnum() or char == " "]).rstrip()
    image.save(f"card_box/{title}.png")


Clearing card_images folder...


In [7]:
def generate_physical_cards():
    # choose a random card from the card deck
    card = random.choice(card_deck)
    print(card)

    # get the summary for the card
    summary = card['summary'][1] if isinstance(card['summary'], list) else card['summary']

    # summarize the summary if it is a string, or convert it to a string if it is a list
    if isinstance(summary, str):
        summary = summarize_text(summary, 2)
    if isinstance(summary, list):
        summary = ' '.join(summary)

    # calculate the points for the card
    points = card['point_value']

    # generate the card image
    generate_card(str(card['title']), summary, points=points)

    # iterate through the card deck and generate card images for each card
    for card in tqdm(card_deck):
        # get the title and summary for the card
        title = card['title']
        summary = card['summary'][1] if isinstance(card['summary'], list) else card['summary']
        sentence_count = 5
        # summarize the summary if it is a string, or convert it to a string if it is a list
        if isinstance(summary, str):
            # if the summarized text fills more than 3/4 the height of the card, summarize it again with one fewer sentence. This is to prevent the text from overflowing the card.
            # max at font size 20 
            while len(summary) > 25 * 40:# and sentence_count > 1:
                summary = summarize_text(summary, 1)
                sentence_count -= 1
        if isinstance(summary, list):
            summary = ' '.join(summary)

        # calculate the points for the card
        points = card['point_value']

        # generate the card image
        generate_card(str(card['title']), summary, points=points)

print("Initialized process, and ready to generate physical cards...")


Initialized process, and ready to generate physical cards...


# 25 lines 

In [8]:

generate_physical_cards()
print("Done")

{'title': 'Back to the Future', 'summary': 'Back to the Future, Back to the Future is a 1985 American science fiction film directed by Robert Zemeckis, and written by Zemeckis and Bob Gale. It stars Michael J. Fox, Christopher Lloyd, Lea Thompson, Crispin Glover, and Thomas F. Wilson. Set in 1985, the story follows Marty McFly Fox, a teenager accidentally sent back to 1955 in a time-traveling DeLorean automobile built by his eccentric scientist friend Emmett Doc Brown Lloyd. While in the past, Marty inadvertently prevents his future parents from falling in lovethreatening his existenceand is forced to reconcile the pair and somehow get back to the future.\\nGale and Zemeckis conceived the idea for Back to the Future in 1980. They were desperate for a successful film after numerous collaborative failures, but the project was rejected over 40 times by various studios because it was not considered raunchy enough to compete with the successful comedies of the era. A development deal was se

 22%|██▏       | 1470/6803 [00:42<02:41, 33.08it/s]